In [5]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy
import scipy.stats as stats
import math

# Plot settings
plt.rcParams['figure.figsize'] = [24, 8]

# Load the dataset
df = pd.read_csv('../dataset/full_data_flightdelay.csv', dtype='unicode')

<figure>
  <IMG SRC="https://www.colorado.edu/cs/profiles/express/themes/cuspirit/logo.png" WIDTH=50 ALIGN="right">
</figure>

# Semester Project - Part 1
*CSPB 3022 Data Science Algorithms - Spring 2022*

*Author: Thomas Cochran*

## Project Topic

**Project goal:**

The goal of this project is to create a classifier that predicts whether a commercial airline flight will be delayed given a set of departure conditions. 

**Type of problem:**

This is a binary classification problem with target classes: `delayed` and `not delayed`. The target class `delayed` is defined as any flight whose departure time exceeds 15 minutes.

**Project motivation:**

Many of us have found ourselves sitting on a boarded plane, probably in a coach middle seat, wondering: why hasn't this plane taken off yet? It would be nice to mentally prepare ourselves for this situation, especially if the person in front of us has already put their seat in full recline mode. This is one motivation for working on this project. An additional motivation is the ability to plan our travel a little more effeciently by knowing how likely a departure delay will occur.


## Dataset

**Source:**

The dataset used in this project is from kaggle ([source](https://www.kaggle.com/datasets/threnjen/2019-airline-delays-and-cancellations)). It consists of a single file located in `../dataset/full_data_flightdelay.csv` which contains 1.27 GB of domestic departure data and associated weather for the year of 2019 that has been merged from the following primary sources:

1. Bureau of Transportation statistics: [Link](https://www.transtats.bts.gov/databases.asp?Z1qr_VQ=E&Z1qr_Qr5p=N8vn6v10&f7owrp6_VQF=D)
2. National Centers for Environmental Information (NOAA): [Link](https://www.ncdc.noaa.gov/cdo-web/datasets)

**Description:**

Departure data from the Bureau of Transportation statistics consists of monthly performance reports that contain a plethora of features for domestic departure flights, including:

* Day of week
* Airline carrier name
* Age of the departing aircraft

The NOAA data is merged with the above departure data which adds some potentially interesting features that may contribute to delays, such as:

* Departure snowfall and percipitation in inches
* Departure temperature
* Departure wind speed

In total, the dataset lists 6,489,062 domestic departures for the year 2019. Each departure has 26 features consisting of 8 categorical and 18 numerical. A description of each feature can be found in the file `../dataset/documentation.md`.

In [8]:
# Brief descriptive statistics for the dataset
df.describe()

,MONTH,DAY_OF_WEEK,DEP_DEL15,DEP_TIME_BLK,DISTANCE_GROUP,SEGMENT_NUMBER,CONCURRENT_FLIGHTS,NUMBER_OF_SEATS,CARRIER_NAME,AIRPORT_FLIGHTS_MONTH,...,PLANE_AGE,DEPARTING_AIRPORT,LATITUDE,LONGITUDE,PREVIOUS_AIRPORT,PRCP,SNOW,SNWD,TMAX,AWND
count,6489062,6489062,6489062,6489062,6489062,6489062,6489062,6489062,6489062,6489062,...,6489062,6489062,6489062,6489062,6489062,6489062,6489062,6489062,6489062,6489062
unique,12,7,2,19,11,15,107,82,17,969,...,33,96,96,96,356,305,75,28,124,130
top,8,5,0,0800-0859,2,2,10,143,Southwest Airlines Co.,35256,...,11,Atlanta Municipal,33.641,-84.427,NONE,0.0,0.0,0.0,89.0,7.61
freq,579532,969141,5261694,452391,1547851,1505083,186040,963805,1296329,35256,...,487576,392603,392603,392603,1449009,4554778,6326795,6316391,167342,254086


## Data cleaning and Exploratory Data Analysis (EDA):

TODO